# Snowflake SQL API Demo

There are several ways to run Snowflake queries from other programming languages, but the most versatile one is the Snowflake SQL API. This API is a REST web service that can be accessed from any tool or programming language that support HTTP interactions like GET and POST.

In this demonstration, we'll use the Snowflake SQL API to create a table and load some data to it using our DATA5035_USER user account.

## Key Pair Authentication

While interacting with Snowflake using a user name and password is possible (we've done it before), a more secure was is to create and use a secure public / private key pair as shown in the instructions here:
* [Setting up key pair authentication](https://docs.snowflake.com/en/user-guide/key-pair-auth#configuring-key-pair-authentication)
* [Using key pair authentication with SQL API](https://docs.snowflake.com/en/developer-guide/sql-api/authenticating#using-key-pair-authentication)

In [ ]:
# Install the pyjwt package if needed
!pip install pyjwt

In [ ]:
# Function to generate a JWT Token

from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import PublicFormat
from cryptography.hazmat.backends import default_backend
from datetime import timedelta, timezone, datetime
import base64
import hashlib
import jwt

def gen_token(account:str, user:str, private_key_file:str):
    """
    account - Snowflake account name in ORGANZIATION-ACCOUNT format
    user - Snowflake user name
    private_key_file - Location and name of file with private key
    """

    # Use uppercase for the account identifier and user name.
    account = account.upper()
    user = user.upper()
    qualified_username = account + "." + user

    # Get the current time in order to specify the time when the JWT was issued and the expiration time of the JWT.
    now = datetime.now(timezone.utc)

    # Specify the length of time during which the JWT will be valid. You can specify at most 1 hour.
    lifetime = timedelta(minutes=59)

    # Load the private key from the specified file.
    with open(private_key_file, 'rb') as pem_in:
        pemlines = pem_in.read()
        private_key = load_pem_private_key(pemlines, None, default_backend())


    # Create public key fingerprint
    public_key_raw = private_key.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo)

    # Get the sha256 hash of the raw bytes.
    sha256hash = hashlib.sha256()
    sha256hash.update(public_key_raw)

    # Base64-encode the value and prepend the prefix 'SHA256:'.
    public_key_fp = 'SHA256:' + base64.b64encode(sha256hash.digest()).decode('utf-8')


    # Create the payload for the token.
    payload = {

        # Set the issuer to the fully qualified username concatenated with the public key fingerprint (calculated in the  previous step).
        "iss": qualified_username + '.' + public_key_fp,

        # Set the subject to the fully qualified username.
        "sub": qualified_username,

        # Set the issue time to now.
        "iat": now,

        # Set the expiration time, based on the lifetime specified for this object.
        "exp": now + lifetime
    }

    # Generate the JWT. private_key is the private key that you read from the private key file in the previous step when you generated the public key fingerprint.
    encoding_algorithm="RS256"
    token = jwt.encode(payload, key=private_key, algorithm=encoding_algorithm)

    # If you are using a version of PyJWT prior to 2.0, jwt.encode returns a byte string, rather than a string.
    # If the token is a byte string, convert it to a string.
    if isinstance(token, bytes):
        token = token.decode('utf-8')

    return token

In [ ]:
# Enter your own Snowflake account information, user, and private key file location
ACCOUNT = "ORG-ACCOUNT"
USERNAME = "DATA5035_USER"
PRIVATE_KEY_FILE = "/path/to/data5035_user.p8"

token = gen_token(ACCOUNT, USERNAME, PRIVATE_KEY_FILE)


Put this content into a file named query.json

```json
{
  "statement": "select count(1) from data5035.public.donations",
  "timeout": 60,
  "database": "DATA5035",
  "schema": "PUBLIC",
  "warehouse": "DATA5035_WH",
  "role": "DATA5035_ROLE"
}
```

Then you can run a command like this to test the JWT you created

```bash
curl -i -X POST \
    -H "Content-Type: application/json" \
    -H "Authorization: Bearer <jwt>" \
    -H "Accept: application/json" \
    -H "User-Agent: myApplicationName/1.0" \
    -H "X-Snowflake-Authorization-Token-Type: KEYPAIR_JWT" \
    -d "@query.json" \
    "https://fullurl.snowflakecomputing.com/api/v2/statements"
```


In [ ]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token}',
    'Accept': 'application/json',
    'User-Agent': 'Python/13.x',
    'X-Snowflake-Authorization-Token-Type': 'KEYPAIR_JWT'
}

response = requests.post(
    headers=headers,
    url='https://fullurl.snowflakecomputing.com/api/v2/statements',
    data=open('query.json').read()
)

In [ ]:
response.json()

In [ ]:
response.json().keys()

In [ ]:
response.json()['data']

In [ ]:
import pandas as pd
df = pd.DataFrame(
    response.json()['data'],
    columns=[x['name'] for x in response.json()['resultSetMetaData']['rowType']]
)

In [ ]:
df